# LRU算法

### 思路

用一个 dict 做哈希表： key -> 双向链表节点，保证 O(1) 找到 key 对应的节点。

用一个双向链表维护“使用顺序”： 头部是最近使用，尾部是最久未使用；用 dummy head / tail 简化操作。

访问或更新某个 key 时： 从哈希表拿到节点，将它从原位置删除，再插到链表头（变成“最近使用”）。

插入新 key 时： 建一个新节点，插到链表头，并写入 dict；如果超出容量，就从链表尾删掉最久未使用的节点，同时从 dict 删掉对应 key。

**get 返回值 + 移动到头，put 更新或新建节点 + 必要时淘汰尾部，整个过程所有操作都保持 O(1)。

### 代码

In [6]:
# LRU Cache 实现 + 调试打印

class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None


class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}  # key -> Node

        # 双向链表：dummy head / tail
        self.head = Node(0, 0)
        self.tail = Node(0, 0)
        self.head.next = self.tail
        self.tail.prev = self.head

    # ---------- 链表基础操作 ----------

    def _remove(self, node: Node):
        """从链表中删掉一个节点（O(1））"""
        prev_node = node.prev
        next_node = node.next
        prev_node.next = next_node
        next_node.prev = prev_node

    def _add_to_head(self, node: Node):
        """把节点插入到 head 后面，作为最近使用（O(1））"""
        node.prev = self.head
        node.next = self.head.next

        self.head.next.prev = node
        self.head.next = node

    def _move_to_head(self, node: Node):
        """节点被使用后，移动到头部"""
        self._remove(node)
        self._add_to_head(node)

    def _remove_tail(self) -> Node:
        """删除链表尾部（最久未使用的那个节点），并返回它"""
        node = self.tail.prev
        self._remove(node)
        return node

    # ---------- LRU 对外接口 ----------

    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1

        node = self.cache[key]
        self._move_to_head(node)
        return node.value

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            # 已存在：更新 value + 移到头部
            node = self.cache[key]
            node.value = value
            self._move_to_head(node)
        else:
            # 不存在：新建节点 + 放头部
            node = Node(key, value)
            self.cache[key] = node
            self._add_to_head(node)

            # 超出容量：删尾部最旧节点
            if len(self.cache) > self.capacity:
                tail_node = self._remove_tail()
                del self.cache[tail_node.key]

    # ---------- 调试用：打印当前链表顺序 ----------
    def debug_print(self, msg=""):
        print(f"\n[{msg}] 当前 LRU 顺序（head -> tail）：")
        cur = self.head.next
        items = []
        while cur is not self.tail:
            items.append(f"{cur.key}:{cur.value}")
            cur = cur.next
        print("  " + "  <->  ".join(items) if items else "  (空)")


# ---------- 在 Notebook 里演示 ----------

lru = LRUCache(2)
print("创建 LRUCache(capacity=2)")

lru.put(1, 1)
lru.debug_print("put(1, 1)")

lru.put(2, 2)
lru.debug_print("put(2, 2)")

print("\nget(1) 返回：", lru.get(1))
lru.debug_print("get(1) 之后")

lru.put(3, 3)  # 这一步会淘汰 key=2
lru.debug_print("put(3, 3)（容量满，淘汰最久未使用的 2）")

print("\nget(2) 返回：", lru.get(2))  # -1，因为 2 已被淘汰

lru.put(4, 4)  # 淘汰 key=1
lru.debug_print("put(4, 4)（再次淘汰最久未使用的 1）")

print("\nget(1) 返回：", lru.get(1))  # -1
print("get(3) 返回：", lru.get(3))  # 3
print("get(4) 返回：", lru.get(4))  # 4
lru.debug_print("最终状态")


创建 LRUCache(capacity=2)

[put(1, 1)] 当前 LRU 顺序（head -> tail）：
  1:1

[put(2, 2)] 当前 LRU 顺序（head -> tail）：
  2:2  <->  1:1

get(1) 返回： 1

[get(1) 之后] 当前 LRU 顺序（head -> tail）：
  1:1  <->  2:2

[put(3, 3)（容量满，淘汰最久未使用的 2）] 当前 LRU 顺序（head -> tail）：
  3:3  <->  1:1

get(2) 返回： -1

[put(4, 4)（再次淘汰最久未使用的 1）] 当前 LRU 顺序（head -> tail）：
  4:4  <->  3:3

get(1) 返回： -1
get(3) 返回： 3
get(4) 返回： 4

[最终状态] 当前 LRU 顺序（head -> tail）：
  4:4  <->  3:3


### 类似题目（707. 设计链表）

### 思路

使用 dummy head 作为虚拟头节点，避免对链表头部插入/删除做特殊判断。

用 size 记录链表长度，便于判断 index 是否越界，并支持快速判断是否在尾部插入。

get(index)：从 dummy head 往后走 index+1 步，返回对应节点的值，越界返回 -1。

addAtIndex(index, val)：先找到 index 的前一个节点，再进行插入（改变指针方向）；index > size 不插入。

deleteAtIndex(index)：找到 index 的前一个节点，将其 next 跳过目标节点，实现 O(1) 删除。

### 代码

In [12]:
# 707. Design Linked List — 可执行版本（含打印）

class Node:
    def __init__(self, val=0):
        self.val = val
        self.next = None


class MyLinkedList:

    def __init__(self):
        self.head = Node(0)   # dummy head
        self.size = 0

    def get(self, index: int) -> int:
        if index < 0 or index >= self.size:
            return -1
        
        curr = self.head
        for _ in range(index + 1):  # 走 index+1 步，因为 dummy 占一格
            curr = curr.next
        return curr.val

    def addAtHead(self, val: int) -> None:
        self.addAtIndex(0, val)

    def addAtTail(self, val: int) -> None:
        self.addAtIndex(self.size, val)

    def addAtIndex(self, index: int, val: int) -> None:
        if index < 0:
            index = 0
        if index > self.size:
            return
        
        prev = self.head
        for _ in range(index):       # 找到 index 前一个节点 prev
            prev = prev.next
        
        node = Node(val)
        node.next = prev.next
        prev.next = node
        self.size += 1

    def deleteAtIndex(self, index: int) -> None:
        if index < 0 or index >= self.size:
            return
        
        prev = self.head
        for _ in range(index):       # 找到 index 前一个节点 prev
            prev = prev.next
        
        prev.next = prev.next.next   # 跳过 index 位置节点
        self.size -= 1


# ---------- 测试输出（Notebook 可见） ----------
def print_list(ll: MyLinkedList, msg=""):
    print(f"\n{msg}")
    cur = ll.head.next
    res = []
    while cur:
        res.append(cur.val)
        cur = cur.next
    print("链表 =", res)


ll = MyLinkedList()

ll.addAtHead(1)
print_list(ll, "addAtHead(1)")

ll.addAtTail(3)
print_list(ll, "addAtTail(3)")

ll.addAtIndex(1, 2)
print_list(ll, "addAtIndex(1, 2)")

print("\nget(1) =", ll.get(1))

ll.deleteAtIndex(1)
print_list(ll, "deleteAtIndex(1)")


addAtHead(1)
链表 = [1]

addAtTail(3)
链表 = [1, 3]

addAtIndex(1, 2)
链表 = [1, 2, 3]

get(1) = 2

deleteAtIndex(1)
链表 = [1, 3]
